## GNN Explainer

Here we visualize the explainer results and analyze specific graph theoretic properties for datatset `syn5`

In [1]:
import numpy as np
import os
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
logdir = '../log/'
# Here all the required .npy files are directly under log
expdir = ''

In [3]:
# Load produced masks
dirs = os.listdir(os.path.join(logdir, expdir))
dirs

['enron_base_h20_o20',
 'grad',
 'graph',
 'mask',
 'masked_adj_syn1_base_h20_o20_explainnode_idx_400graph_idx_-1.npy',
 'masked_adj_syn1_base_h20_o20_explainnode_idx_405graph_idx_-1.npy',
 'masked_adj_syn1_base_h20_o20_explainnode_idx_410graph_idx_-1.npy',
 'masked_adj_syn1_base_h20_o20_explainnode_idx_415graph_idx_-1.npy',
 'masked_adj_syn1_base_h20_o20_explainnode_idx_420graph_idx_-1.npy',
 'masked_adj_syn1_base_h20_o20_explainnode_idx_425graph_idx_-1.npy',
 'masked_adj_syn1_base_h20_o20_explainnode_idx_430graph_idx_-1.npy',
 'masked_adj_syn1_base_h20_o20_explainnode_idx_435graph_idx_-1.npy',
 'masked_adj_syn1_base_h20_o20_explainnode_idx_440graph_idx_-1.npy',
 'masked_adj_syn1_base_h20_o20_explainnode_idx_445graph_idx_-1.npy',
 'masked_adj_syn1_base_h20_o20_explainnode_idx_450graph_idx_-1.npy',
 'masked_adj_syn1_base_h20_o20_explainnode_idx_455graph_idx_-1.npy',
 'masked_adj_syn1_base_h20_o20_explainnode_idx_460graph_idx_-1.npy',
 'masked_adj_syn1_base_h20_o20_explainnode_idx_465gr

In [4]:
masks = []

# This would print all the files and directories
for file in dirs:
    # Check if file extension is ".npy" which are
    # numpy binary files to represent large data
    if file.split('.')[-1] == 'npy' and file.split('_')[2] == 'syn5':
        # print(file)
        masks.append(file)
len(masks)

60

In [5]:
def show_adjacency_full(mask, ax=None):
    """Plot full adjacency matrix of the mask

    Args:
        mask (str): Filename containing the adjacency matrix of the mask
        ax (Axes class object, optional): Axis. Defaults to None.

    Returns:
        numpy matrix: Full adjacency matrix of the mask
    """
    # Obtain adjacency matrix from the filename sent in var "mask"
    adj = np.load(os.path.join(logdir, expdir, mask), allow_pickle=True)
    # if ax is None:
    #     plt.figure()
    #     plt.imshow(adj);
    # else:
    #     ax.imshow(adj)
    return adj

In [6]:
def filter_adj(adj,threshold=0.8):
    """Filter out the values in adjacency matrix that are greater than threshold. Fix the others as 0

    Args:
        adj (numpy matrix): Adjacency matrix of mask that we have to filter
        threshold (float): Filter value

    Returns:
        numpy matrix: Filtered adjacency matrix
    """
    filt_adj = adj.copy()
    filt_adj[adj<threshold] = 0
    return filt_adj

### Analyzing Graph theoretic properties of each mask

Here we measure the following quantities:
- Nodes: |V|
- Edges: |E|
- Avg degree
- Diameter of the graph
- Sparsity measure: edge density = |E|/C(|V|,2)
- Node centrality measure: Betweenness centrality

In [12]:
def check_symmetric(a, rtol=1e-05, atol=1e-08):
    return np.allclose(a, a.T, rtol=rtol, atol=atol)

def normalize(A):
    scale_factor = A.max() - A.min()
    B = np.ones_like(A)*A.min()

    A = (A - B)/scale_factor
    return A

def centrality(adj):
    # Get the unweighted adjacency matrix
    adj1 = adj.copy()
    adj1[adj>0] = 1

    # Get the unweighted graph
    unweighted_G = nx.from_numpy_matrix(np.matrix(adj1))

    # For now we use edge-path centrality later check
    # Information centrality metric
    ebc = nx.edge_betweenness_centrality(unweighted_G)

    # Create a symmetric matrix A with A[i][j] is the betweeness score of edge (i,j)
    A = np.zeros_like(adj1)
    v = unweighted_G.number_of_nodes()
    for i in range(v):
        for j in range(i):
            if (j,i) in ebc.keys():
                # The keys in ebc are (a,b) with a < b
                A[i][j] = A[j][i] = ebc[(j,i)]
    
    # Now that we have got A, we see if the importance given by GNNExplainer
    # has some relation with the centrality scores in A
    A1 = normalize(A)
    B1 = normalize(adj)
    # For coherence check the following two links:
    # 1) https://www.geeksforgeeks.org/cosine-similarity-calculation-between-two-matrices-in-matlab/
    # 2) https://math.stackexchange.com/questions/507742/distance-similarity-between-two-matrices#:~:text=If%20we%20have%20two%20matrices,squares%20of%20all%20singular%20values.
    coherence = 1/(np.linalg.norm(A1 - B1,2))**2

    return coherence,A1, B1

def graph_prop(adj):
    # Edge weighted graph
    weighted_G = nx.from_numpy_matrix(np.matrix(adj))

    v = weighted_G.number_of_nodes()
    e = weighted_G.number_of_edges()
    avg_degree = float('%.3f'%(2*e/v))
    diameter = nx.diameter(weighted_G)
    sparsity = float('%.3f'%(2*e/(v*(v-1))))
    coherence = float('%.3f'%(centrality(adj)))

    return [v,e,avg_degree,diameter,sparsity,coherence]

In [8]:
data = []

for m in masks:
    # Full adjacency matrix
    adj = show_adjacency_full(m)
    # Get the metrics of each mask
    data.append(graph_prop(adj))

In [9]:
import pandas as pd

# Convert the above data into a data frame
df = pd.DataFrame(data, columns=["Order","Size","Average Degree","Diameter","Sparsity","Coherence"])


df.describe()

,Order,Size,Average Degree,Diameter,Sparsity,Coherence
count,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000
mean,15.783333,17.366667,2.168150,5.250000,0.175900,0.916967
std,7.216017,8.447177,0.267956,0.985058,0.071899,0.944089
min,6.000000,5.000000,1.667000,3.000000,0.051000,0.229000
25%,10.750000,12.000000,1.891750,4.000000,0.120250,0.296750
50%,15.000000,16.000000,2.273500,6.000000,0.156000,0.501500
75%,18.250000,21.000000,2.390250,6.000000,0.227000,0.954750
max,43.000000,46.000000,2.600000,6.000000,0.333000,3.762000


In [21]:
# Normalize the Coherence
coh_int = list(df["Coherence"])
coh = [float(i)/max(coh_int) for i in coh_int]

df = df.replace(coh_int, coh)
df.describe()

,Order,Size,Average Degree,Diameter,Sparsity,Coherence
count,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000
mean,15.783333,17.366667,2.168150,5.250000,0.158891,0.243744
std,7.216017,8.447177,0.267956,0.985058,0.069145,0.250954
min,6.000000,5.000000,1.667000,3.000000,0.051000,0.060872
25%,10.750000,12.000000,1.891750,4.000000,0.112250,0.078881
50%,15.000000,16.000000,2.273500,6.000000,0.143000,0.133307
75%,18.250000,21.000000,2.390250,6.000000,0.192000,0.253788
max,43.000000,46.000000,2.600000,6.000000,0.333000,1.000000
